In [ ]:
import pandas as pd

path_file_0 = '/content/Planilla_EP2.csv'
path_file_1 = '/content/Asistencia_Electiva_de_Programacion_4.csv'
path_file_2 = '/content/Asistencia_Electiva_de_Programacion_5.csv'
path_file_3 = '/content/Asistencia_Electiva_de_Programacion_6.csv'

""" Función para abrir archivo CSV con codificación UTF8 y separador ; """
def openFile(file_path):
  return pd.read_csv(file_path, sep=';',encoding='utf-8')

""" Función para corregir indices en la numeración"""
def fixIndex(column):
  temp = column.reset_index()
  return temp.drop(['index'], axis=1)

""" Función para remover acentos en el DataFrame obtenido por parámetro"""
def deleteAccents(dataframe):
  string_cols = dataframe.columns[dataframe.dtypes == 'object']
  dataframe[string_cols] = dataframe[string_cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
  return dataframe

""" Función que extrae los valores de una columna específica"""
def extractNamePlanilla(file):
  return file[['Apellidos y Nombres del Estudiante']]

""" Función que elimina elementos vacios de la columna recibida por parámetro"""
def cleanData(column):
  return column.drop(index=([21,22]))

""" Función que ordena la columna recibida por parámetro alfabeticamente"""
def sortColumnPlanilla(column):
  return column.sort_values('Apellidos y Nombres del Estudiante', ascending= True)

""" Función que renombra la columna recibida por parámetro"""
def renameColumn(column):
  return column.rename(columns={'Apellidos y Nombres del Estudiante' : 'APELLIDOS_Y_NOMBRES'})
""" Función que extrae las columnas necesarias de el archvo que recibe por parámetro"""
def extractData(file):
  temp = file[['Marca temporal','Escriba Nombres. ','Escriba Apellidos.']]
  return temp.drop(['Marca temporal'], axis=1)

""" Función que renombra las columnas obtenidas del archivo recibido como parámetro"""
def renameColumnsCommonFile(file):
  return file.rename(columns={'Escriba Nombres. ':'Nombres','Escriba Apellidos.' : 'Apellidos'})

""" Función que une los nombres y apellidos sin dobles espacios del del archivo recibido como parámetro"""
def joinNameSurname(file):
  file['APELLIDOS_Y_NOMBRES'] = file['Apellidos'].str.strip() + ' ' + file['Nombres'].str.strip()
  file = file.drop(['Apellidos','Nombres'], axis=1)
  file = file.reindex(columns=['APELLIDOS_Y_NOMBRES'])           
  return file

""" Función que ordena alfabeticamente el contenido del archivo recibido como parámetro"""
def sortColumnData(file):
  file = file.sort_values('APELLIDOS_Y_NOMBRES', ascending= True)
  return file

""" Función que establece en mayúsculas el contenido del archivo recibido como parámetro"""
def setUpperCase(file):
  file ['APELLIDOS_Y_NOMBRES'] = file['APELLIDOS_Y_NOMBRES'].str.upper()
  return file

""" Función que espera recibir un archivo de planilla como parámetro y ejecutar las transformaciones necesarias"""
def planilla(planilla_path):
  temp = renameColumn(fixIndex(sortColumnPlanilla(cleanData(extractNamePlanilla(openFile(planilla_path))))))
  temp['APELLIDOS_Y_NOMBRES'] = temp['APELLIDOS_Y_NOMBRES'].str.strip()
  return pd.DataFrame.from_dict(temp)

""" Función que espera recibir un archivo de asistencia como parámetro y ejecutar las transformaciones necesarias"""
def commonFile(commonFile_path):
  temp = setUpperCase(fixIndex(sortColumnData(joinNameSurname(renameColumnsCommonFile(extractData(openFile(commonFile_path)))))))
  temp['APELLIDOS_Y_NOMBRES'] = temp['APELLIDOS_Y_NOMBRES'].str.strip()
  return pd.DataFrame.from_dict(temp)

""" Se define el DataFrame para la comparación resultante"""
df_final = pd.DataFrame()

""" Se define el DataFrame para la planilla"""
df_planilla = deleteAccents(planilla(path_file_0))

""" Se define los DataFrames para las listas de asistencia"""
df0_asistencia = deleteAccents(commonFile(path_file_1))
df1_asistencia = deleteAccents(commonFile(path_file_2))
df2_asistencia = deleteAccents(commonFile(path_file_3))

""" Se compara la planilla y las listas de asistencias, y el resultado se pone dentro del DataFrame final"""
df_final['APELLIDOS_Y_NOMBRES'] = df_planilla['APELLIDOS_Y_NOMBRES']
df_final['06/07/2022'] = df_planilla.APELLIDOS_Y_NOMBRES.isin(df0_asistencia.APELLIDOS_Y_NOMBRES)
df_final['06/09/2022'] = df_planilla.APELLIDOS_Y_NOMBRES.isin(df1_asistencia.APELLIDOS_Y_NOMBRES) 
df_final['06/14/2022'] = df_planilla.APELLIDOS_Y_NOMBRES.isin(df2_asistencia.APELLIDOS_Y_NOMBRES) 

""" Se intercambian los valores True por Si y False por No"""
bool_cols = df_final.columns[df_final.dtypes == 'bool']
df_final[bool_cols] = df_final[bool_cols].replace({True: 'Si', False: 'No'})

""" Se define el DataFrame de la cuenta de asistencias"""
df_count = pd.DataFrame()

""" Se ubican los valores en el DataFrame"""
df_count['06/07/2022'] = df0_asistencia.count()
df_count['06/09/2022'] = df1_asistencia.count()
df_count['06/14/2022'] = df2_asistencia.count()
